In [1]:
# 2️⃣ Importar bibliotecas necessárias
import pandas as pd
from google.colab import files
from google.cloud import bigquery
from google.colab import auth

In [2]:
# 1️⃣ Fazer upload do arquivo CSV do seu computador

uploaded = files.upload()  # Abre uma janela para você selecionar o arquivo
csv_filename = list(uploaded.keys())[0]  # Nome do arquivo enviado

print(f"📂 Arquivo '{csv_filename}' carregado com sucesso!")

Saving mmm_sample_data.csv to mmm_sample_data.csv
📂 Arquivo 'mmm_sample_data.csv' carregado com sucesso!


In [3]:
# 3️⃣ Autenticar no Google Cloud (usando login manual OAuth)
auth.authenticate_user()

In [4]:
# 4️⃣ Definir configurações do projeto e dataset no BigQuery
PROJECT_ID = "meu-projeto-dbt"  # Substitua pelo seu project_id
DATASET_ID = "sample_data"
TABLE_ID = "marketing_data"

In [5]:
# 5️⃣ Carregar o arquivo CSV para um dataframe pandas
df = pd.read_csv(csv_filename)

In [6]:
# 6️⃣ Criar cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

In [11]:
# Criar dataset se não existir
full_dataset_id = f"{PROJECT_ID}.{DATASET_ID}"
dataset = bigquery.Dataset(full_dataset_id)
dataset.location = "US"

try:
    dataset = client.create_dataset(dataset, exists_ok=True)
    print(f"✅ Dataset '{full_dataset_id}' criado ou já existente.")
except Exception as e:
    print(f"⚠️ Erro ao criar o dataset: {e}")

✅ Dataset 'meu-projeto-dbt.sample_data' criado com sucesso!


In [12]:
# 7️⃣ Configurar a carga da tabela
job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Detecta automaticamente os tipos de dados
    write_disposition="WRITE_TRUNCATE",  # Sobrescreve a tabela se já existir
)

In [13]:
# 8️⃣ Carregar os dados para o BigQuery
table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Espera a conclusão

print(f"✅ Dados carregados com sucesso na tabela `{table_ref}`!")

✅ Dados carregados com sucesso na tabela `meu-projeto-dbt.sample_data.marketing_data`!


In [17]:
# 9️⃣ Verificar se os dados foram carregados corretamente
query = f"SELECT COUNT(*) AS total_linhas FROM `{table_ref}`"
result = client.query(query).to_dataframe()
print(f"📊 Total de linhas carregadas: {result['total_linhas'][0]}")


print("")

query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}` LIMIT 5"
result = client.query(query).to_dataframe()
print("📊 Primeiras 5 linhas da tabela carregada:")
result

📊 Total de linhas carregadas: 804

📊 Primeiras 5 linhas da tabela carregada:


,date,tv,radio,ooh,meta,google,tiktok,digital,sales,holiday
0,2023-04-27,0.00,NaN,1084.20,3796.61,7159.81,4019.28,2106.97,7693449.97,0.0
1,2023-08-25,4055.11,NaN,1012.66,4035.08,4607.99,2801.70,1784.53,7512609.44,0.0
2,2023-10-14,3640.98,NaN,934.71,2275.59,4254.67,2727.94,1627.36,5274272.74,0.0
3,2023-11-17,6816.80,NaN,752.11,3965.98,9265.49,4155.98,2098.45,49283224.10,0.0
4,2024-03-19,9435.95,NaN,1296.58,5003.28,10658.48,8675.10,2588.44,24741935.76,0.0
